# Importando Bibliotecas

In [1]:
import scrapy
import requests
import csv
import time
import random
import re
import pandas as pd
import os

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [3]:
verde = '\033[92m'
amarelo = '\033[93m'
vermelho = '\033[91m'
azul = '\033[94m'
fim = '\033[0m'

dados_coletados_geral = []

In [33]:

def persist(dados, arquivo='imovel.csv'):
    df = pd.DataFrame(dados)
    
    # Se o arquivo já existir, adiciona sem cabeçalho
    if os.path.exists(arquivo):
        df.to_csv(arquivo, mode='a', header=False, index=False, encoding='utf-8')
    else:
        df.to_csv(arquivo, index=False, encoding='utf-8')

    return []


# Imobiliaria Giareta-Julio

In [34]:
print(f'{azul}\n------------------Iniciado Imobiliária Giaretta------------------{fim}')

user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/128.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.5 Safari/605.1.15',
]

pagina = 1
contador_imoveis = 0

while True:
    url_geral = f'https://www.imobiliariagiaretta.com.br/locacao/todos/todas/todos/todos----todos/todos/todos---todos/lin/{pagina}?busca=1&exi=col&ord=asc&suites=0&'
    random_user_agent = random.choice(user_agents)
    headers = {"User-Agent": random_user_agent}

    try:
        html_content = requests.get(url_geral, headers = headers, timeout=20).text
    except (requests.exceptions.ReadTimeout, requests.exceptions.ConnectionError) as e:
        print(f"{amarelo}Erro de conexão/timeout na página {pagina}. Tentando novamente em 7 segundos...{fim}")
        time.sleep(7)
        continue

    time.sleep(random.uniform(2, 5)) 
    response = scrapy.http.HtmlResponse(url = url_geral, body = html_content, encoding = "utf-8")

    imoveis_na_pagina = response.css("div.div-block-71.lin")
    if len(imoveis_na_pagina) == 0:
        break

    print(f"{azul}Coletando página: {pagina}{fim}")

    for imovel in imoveis_na_pagina:
        contador_imoveis += 1

        bairro = imovel.css("div.text-block-5.fullcard::text").get().split()
        bairro = ' '.join(bairro[:-1]).lower()

        titulo = imovel.css("div.text-block-5.fullcard.titulo::text").get().split(",")[0]

        tipo = None
        if "Barracão" in titulo:
            tipo = "Barracão"
        elif "Terreno" in titulo:
            tipo = 'Terreno'
        elif "Comercial" in titulo:
            tipo = "Sala Comercial"
        elif "Casa" or "Sobrado" or "Apartamento" in titulo:
            tipo = "Residencial"
        else:
            tipo = "Não Identificado"

        dados_coletados_geral.append({
            "Titulo do imóvel": titulo,
            "Tipo do imóvel": tipo,
            "Preço": None,
            "Valor Condomínio": None,
			"Bairro": bairro,
    		"Link para o anúncio": imovel.css("a.link-imovel::attr(href)").get(), 
            "Origem": 'Giaretta',
            "Quartos": None,
            "Suítes": None,
            "Área do imóvel": None
        })
        print(f"{contador_imoveis}.{verde} Coletado:", titulo, f'{fim}')
        time.sleep(0.5)
    time.sleep(1)
    pagina += 1

print(f'{azul}Coletados {contador_imoveis} imóveis da Imobiliária Giaretta{fim}')

i = 0
while i <= len(dados_coletados_geral) - 1:
    imovel = dados_coletados_geral[i]

    url = imovel['Link para o anúncio']
    random_user_agent = random.choice(user_agents)

    try:
        html_content = requests.get(url, headers = headers, timeout=10).text
    except (requests.exceptions.ReadTimeout, requests.exceptions.ConnectionError) as e:
        print(f"{amarelo}Erro de conexão/timeout no imóvel {i+1}. Tentando novamente em 5 segundos...{fim}")
        time.sleep(5)
        continue

    html_content= requests.get(url, headers = headers).text
    time.sleep(random.uniform(1, 2)) 
    response = scrapy.http.HtmlResponse(url = url, body = html_content, encoding = "utf-8")

    preco = None
    for dados in response.css("div.grid-soma"):
        preco = dados.xpath('.//div[contains(text(), "Locação:")]/following-sibling::div[1]/text()').get()
        if preco != None:
            preco = int(preco.split()[1].replace('.', '')[:-3])
            imovel["Preço"] = preco
            condominio = dados.xpath('.//div[contains(text(), "Condomínio")]/following-sibling::div[1]/text()').get()
            if condominio != None:
                condominio = int(condominio.split()[1].replace('.', '')[:-3])
                imovel["Valor Condomínio"] = condominio
                break
            else:
                imovel["Valor Condomínio"] = 0
                break
    if preco == None:
        for dados in response.css("div.camp-infos-vallue"):
            preco = dados.xpath('.//div[contains(text(), "R$")]/text()').get()
            if preco != None:
                preco = int(preco.split()[1].replace('.', '')[:-3])
                imovel["Preço"] = preco
                imovel["Valor Condomínio"] = 0
                break

    quartos = None
    for dados in response.css("div.w-col.w-col-6"):
        quartos = dados.xpath('.//div[contains(text(), "Quarto(s)")]/text()').get()
        if quartos != None:
            quartos = int(quartos.split()[0])
            imovel["Quartos"] = quartos
            break
    if quartos == None:  
        imovel["Quartos"] = 0
    
    suites = None
    for dados in response.css("div.w-col.w-col-6"):
        suites = dados.xpath('.//div[contains(text(), "Suíte(s)")]/text()').get()
        if suites != None:
            suites = int(suites.split()[0])
            imovel["Suítes"] = suites
            break
    if suites == None:
        imovel["Suítes"] = 0

    area_imovel = None
    for dados in response.css("div.w-col.w-col-6"):
        area_imovel = dados.xpath('.//div[contains(text(), "Total")]/text()').get()
        if area_imovel != None:
            area_imovel = float(area_imovel.split()[0].replace('.' , '').replace(',', '.'))
            imovel["Área do imóvel"] = area_imovel
            break
    if area_imovel == None:
        for dados in response.css("div.w-col.w-col-6"):
            area_imovel = dados.xpath('.//div[contains(text(), "(m²)")]/text()').get()
            if area_imovel != None:
                area_imovel = float(area_imovel.split()[0].replace('.' , '').replace(',', '.'))
                imovel["Área do imóvel"] = area_imovel
                break

    print(f"{i+1}/{contador_imoveis}{verde} Coletado dados específicos:", dados_coletados_geral[i]["Titulo do imóvel"], f'{fim}')
    time.sleep(0.5)
    i += 1

print(f'\n{azul}------------------Finalizado Imobiliária Giaretta------------------{fim}')


------------------Iniciado Imobiliária Giaretta------------------
Coletando página: 1
1. Coletado: Sala Comercial Para Locação No Edifício Thielke 
2. Coletado: Sala Comercial Para Locação Em Ótima Localização 
3. Coletado: Apartamento Térreo Com Ótimo Custo Para Locação 
4. Coletado: Apartamento Amplo Para Locação 
5. Coletado: Apartamento Para Locação 
6. Coletado: Apartamento Disponível Para Locação 
7. Coletado: Terreno Amplo Disponível Para Locação No Jardim Panorama Ii 
8. Coletado: Apartamento Amplo Para Locação No Edifício San Mar 
9. Coletado: Apartamento Amplo E Bem Localizado Disponível Para Locação 
10. Coletado: Apartamento Para Locação No Edifício Montreal 
11. Coletado: Apartamento Amplo Para Locação No Edifício Podolan 
12. Coletado: Apartamento Com Excelente Localização Para Locação 
13. Coletado: Apartamento Amplo Para Locação 
14. Coletado: Casa Nova Em Ótima Localização Para Locação 
15. Coletado: Sala Comercial Ampla Para Locação Em Excelente Localização 
16. Cole

In [35]:
dados_coletados_geral = persist(dados_coletados_geral)

# Imobiliaria Alianca-Julio

In [39]:
contador_imoveis = 0
pagina = 1

print(f'{azul}------------------Iniciado Imobiliária Aliança------------------{fim}\n')

while True:
    url_geral = f'https://www.imobiliariaaliancatoledo.com.br/filtro/locacao/todos/todas/todos/todos/1/{pagina}'
    random_user_agent = random.choice(user_agents)
    headers = {"User-Agent": random_user_agent}

    try:
        html_content = requests.get(url_geral, headers = headers, timeout=20).text
    except (requests.exceptions.ReadTimeout, requests.exceptions.ConnectionError) as e:
        print(f"{amarelo}Erro de conexão/timeout na página {pagina}. Tentando novamente em 7 segundos...{fim}")
        time.sleep(7)
        continue

    time.sleep(random.uniform(2, 5)) 
    response = scrapy.http.HtmlResponse(url = url_geral, body = html_content, encoding = "utf-8")

    imoveis_na_pagina = response.css("div.div-block-61.teste2")
    if len(imoveis_na_pagina) == 0:
        break

    print(f"{azul}Coletando página: {pagina}{fim}")

    for imovel in imoveis_na_pagina:
        contador_imoveis += 1

        tipo = imovel.css("div.text-block-35::text").get()

        if tipo == 'CASA' or tipo == "APARTAMENTO":
            tipo = "Residencial"
        elif tipo == "ÁREA INDUSTRIAL" or tipo == 'BARRACÃO':
            tipo = "Barracão"
        elif tipo == 'COMÉRCIO' or tipo == 'SALA COMERCIAL':
            tipo = 'Sala Comercial'
        elif tipo == 'TERRENO':
            tipo = 'Terreno'
        else:
            tipo = 'Não Identificado'

        dados_coletados_geral.append({
            "Titulo do imóvel": imovel.css("div.text-block-36::text").get(),
            "Tipo do imóvel": tipo,
            "Preço": None,
            "Valor Condomínio": None,
			"Bairro": None,
    		"Link para o anúncio": imovel.css("a.link-block-13.w-inline-block::attr(href)").get(), 
            "Origem": 'Aliança',
            "Quartos": None,
            "Suítes": None,
            "Área do imóvel": None
        })
        print(f"{contador_imoveis}.{verde} Coletado:", imovel.css("div.text-block-36::text").get(), f'{fim}')
        time.sleep(0.5)
    time.sleep(1)
    pagina += 1

print(f'{azul}Coletados {contador_imoveis} imóveis da Imobiliária Aliança{fim}')

i = len(dados_coletados_geral) - contador_imoveis
contador_atual = 1

while i <= len(dados_coletados_geral) - 1:
    imovel = dados_coletados_geral[i]

    url = imovel['Link para o anúncio']
    random_user_agent = random.choice(user_agents)
    headers = {"User-Agent": random_user_agent}
    
    try:
        html_content = requests.get(url, headers = headers, timeout=10).text
    except (requests.exceptions.ReadTimeout, requests.exceptions.ConnectionError) as e:
        print(f"{amarelo}Erro de conexão/timeout no imóvel {i+1}. Tentando novamente em 5 segundos...{fim}")
        time.sleep(5)
        continue

    time.sleep(random.uniform(1, 2))
    response = scrapy.http.HtmlResponse(url = url, body = html_content, encoding = "utf-8")

    preco = None
    for dados in response.css("div.text-block-16"):
        preco = dados.xpath('./text()').getall()
        for a in preco:
            if '$' in a:
                preco = int(a.split('$')[1].split()[0].replace('.', '')[:-3])
                imovel["Preço"] = preco
                break

    valor_condominio = None
    for dados in response.css("div.text-block-16"):
        valor_condominio = dados.xpath('./div/text()').get()
        if valor_condominio != None:
            valor_condominio = int(valor_condominio.strip().split()[-1].replace('.', '')[:-3])
            if valor_condominio < preco:
                imovel["Valor Condomínio"] = valor_condominio
                break
    if valor_condominio == None:
        imovel["Valor Condomínio"] = 0

    for dados in response.css('div.div-block-29'):
        area = dados.xpath('.//div[contains(text(), "Terreno")]/preceding-sibling::div[1]/text()').get()
        if area != None:
            area = float(area.replace(',','.'))
            imovel["Área do imóvel"] = area
            break
        else:
            area = dados.xpath('.//div[contains(text(), "Útil")]/preceding-sibling::div[1]/text()').get()
            if area != None:
                area = float(area.replace(',','.'))
                imovel["Área do imóvel"] = area
                break
            else:
                area = dados.xpath('.//div[contains(text(), "(m²)")]/preceding-sibling::div[1]/text()').get()
                if area != None:
                    area = float(area.replace(',','.'))
                    imovel["Área do imóvel"] = area
                    break

    quartos = None
    for dados in response.css('div.div-block-29'):
        quartos = dados.xpath('.//div[contains(text(), "Quarto(s)")]/preceding-sibling::div[1]/text()').get()
        if quartos != None:
            quartos = int(quartos)
            imovel["Quartos"] = quartos
            break
    if quartos == None:
        imovel["Quartos"] = 0

    suites = None
    for dados in response.css('div.div-block-29'):
        suites = dados.xpath('.//div[contains(text(), "Suíte(s)")]/preceding-sibling::div[1]/text()').get()
        if suites != None:
            suites = int(suites)
            imovel["Suítes"] = suites
            break
    if suites == None:
        imovel["Suítes"] = 0

    for dados in response.css("div.text-block-19"):
        bairro = dados.xpath('./text()').get().lower()
        imovel["Bairro"] = bairro
        break 

    print(f"{contador_atual}/{contador_imoveis}{verde} Coletado dados específicos de", imovel["Titulo do imóvel"], f'{fim}')
    i += 1
    contador_atual += 1
    time.sleep(0.5)

print(f'\n{azul}------------------Finalizado Imobiliária Aliança------------------{fim}')


------------------Iniciado Imobiliária Aliança------------------



Coletando página: 1
1. Coletado: APARTAMENTO BAIRRO VILA BECKER  
2. Coletado: APARTAMENTO CENTRO  
3. Coletado: APARTAMENTO BAIRRO CENTRO  
4. Coletado: SALA COMERCIAL BAIRRO JARDIM PORTO ALEGRE  
5. Coletado: CASA BAIRRO CENTRO  
6. Coletado: TERRENO BAIRRO CENTRO  
7. Coletado: BARRACÃO BAIRRO CENTRO  
8. Coletado: SALA COMERCIAL NA VILA PIONEIRO.  
9. Coletado: SALA COMERCIAL CENTRO  
Coletando página: 2
10. Coletado: Sobrado para locação, JARDIM PORTO ALEGRE, TOLEDO - PR  
11. Coletado: SOBRADO BAIRRO JARDIM PORTO ALEGRE  
12. Coletado: Sala Comercial para locação, CENTRO, TOLEDO - PR  
13. Coletado: Sala Comercial para locação, JARDIM PORTO ALEGRE, TOLEDO - PR  
14. Coletado: Barracão para fábricas, indústrias, depósitos para locação, em TOLEDO - PR  
15. Coletado: Sala Comercial para locação, CENTRO, TOLEDO - PR  
16. Coletado: TERRENO NO JARDIM PANCERA  
17. Coletado: Sobrado, JARDIM PANCERA, TOLEDO - PR  
18. Coletado: Sala Comercial para locação, CENTRO, TOLEDO - PR  
Coletan

In [ ]:
dados_coletados_geral = persist(dados_coletados_geral)

# Imobiliaria La Selle-Daniel

In [40]:
contador_imoveis = 0
pagina = 1

print(f'{azul}------------------Iniciado Imobiliária La Salle------------------{fim}\n')

while True:
    url_geral = f"https://www.imobiliarialasalle.com.br/filtro/locacao/todos/todas/todos/todos/todos/{pagina}"
    random_user_agent = random.choice(user_agents)
    headers = {"User-Agent": random_user_agent}

    try:
        html_content = requests.get(url_geral, headers = headers, timeout=20).text
    except (requests.exceptions.ReadTimeout, requests.exceptions.ConnectionError) as e:
        print(f"{amarelo}Erro de conexão/timeout na página {pagina}. Tentando novamente em 7 segundos...{fim}")
        time.sleep(7)
        continue

    time.sleep(random.uniform(2, 5)) 
    response = scrapy.http.HtmlResponse(url = url_geral, body = html_content, encoding = "utf-8")

    imoveis_na_pagina = response.css("div.div-block-61.teste2")
    if len(imoveis_na_pagina) == 0:
        break

    print(f"{azul}Coletando página: {pagina}{fim}")

    for imovel in response.css("div.div-block-61.teste2"):
        contador_imoveis += 1

        tipo = (imovel.css("div.text-block-update1::text").get() or "").upper()
        if "BARRACÃO" in tipo:
            tipo = "Barracão"
        elif "Terreno" in tipo:
            tipo = 'Terreno'
        elif "LOJA" or "SALA COMERCIAL" in tipo:
            tipo = "Sala Comercial"
        elif "CASA" or "SOBRADO" or "APARTAMENTO" in tipo:
            tipo = "Residencial"
        else:
            tipo = "Não Identificado"
    
        bairro = (imovel.css("div.text-block-2-update1::text").get() or "")

        preco = int(imovel.css("span.text-span-value::text").get().replace("R$", "").replace(".", "")[:-3])

        link = imovel.css("a.link-block-update1.w-inline-block::attr(href)").get()
        
        titulo_temp = link.split('/')[-2].replace('-', ' ')

        dados_coletados_geral.append({
            "Titulo do imóvel": None,
            "Tipo do imóvel": tipo,
            "Preço": preco,
            "Valor Condomínio": None,
            "Bairro": bairro,
            "Link para o anúncio": link,
            "Origem": "La Salle",
            "Quartos": None,
            "Suíte": None,
            "Área do imóvel": None,
        })




        print(f"{contador_imoveis}.{verde} Coletado: {titulo_temp}{fim}")
        time.sleep(0.5)
    time.sleep(1)
    pagina += 1

print(f'{azul}Coletados {contador_imoveis} imóveis da Imobiliária La Salle{fim}')

i = len(dados_coletados_geral) - contador_imoveis
contador_atual = 1

while i <= len(dados_coletados_geral) - 1:
    imovel = dados_coletados_geral[i]

    url = imovel['Link para o anúncio']
    random_user_agent = random.choice(user_agents)
    headers = {"User-Agent": random_user_agent}

    html = requests.get(url, headers = headers).text
    response = scrapy.http.HtmlResponse(url = url, body = html, encoding = "utf-8")

    try:
        html_content = requests.get(url, headers = headers, timeout=10).text
    except (requests.exceptions.ReadTimeout, requests.exceptions.ConnectionError) as e:
        print(f"{amarelo}Erro de conexão/timeout no imóvel {i+1}. Tentando novamente em 5 segundos...{fim}")
        time.sleep(5)
        continue

    time.sleep(random.uniform(1, 2.5))
    response = scrapy.http.HtmlResponse(url = url, body = html_content, encoding = "utf-8")

    for dados in response.css("div.div-block-19.w-clearfix"):
        imovel["Titulo do imóvel"] = (dados.css("h1.heading-5::text").get() or "")

    for dados in response.css("div.div-block-28"):
        quarto = dados.xpath(".//div[contains(., 'Quarto') or contains (., 'Dormitório')]//text()").re_first(r'(\d+)\s*Quartos?|Dormitórios?') or "0"
        if quarto:
            imovel["Quartos"] = int(quarto)

        suite = dados.xpath(".//div[contains(., 'Suíte')]//text()").re_first(r'(\d+)\s*Suítes?') or "0"
        if suite:
            imovel["Suítes"] = int(suite)

    for dados in response.css("div.div-block-29, div.feature-wrap"):
        area_imovel = (dados.xpath(".//div[contains(., 'm²')]/b/text()").get() or "0").replace(",", ".")[:-2]
        if area_imovel:
            imovel["Área do imóvel"] = float(area_imovel)

    for dados in response.css("div.div-block-32, div.text-block-19"):
        condominio_trecho = dados.xpath(".//*[contains(., 'Valor do condomínio')]//text()").getall() or "0"
        for valor_condominio in condominio_trecho:
            valor = re.search(r'R\$\s*[\d.,]+', valor_condominio)
            if valor: 
                imovel["Valor Condomínio"] = valor.group(0).replace("R$", "")[:-3]

    print(f"{contador_atual}/{contador_imoveis}{verde} Coletado dados específicos de", imovel["Titulo do imóvel"], f'{fim}')
    i += 1
    contador_atual += 1
    time.sleep(0.5)

print(f'\n{azul}------------------Finalizado Imobiliária La Salle------------------{fim}')


------------------Iniciado Imobiliária La Salle------------------

Coletando página: 1
1. Coletado: kitnet disponivel para locacao no jd  coopagro   toledo   pr
2. Coletado: apartamento com 01 quarto para locacao  vila becker   toledo pr
3. Coletado: apartamento para locacao  jardim santa maria  toledo   pr
4. Coletado: apartamento para locacao  jardim santa maria  toledo   pr
5. Coletado: apartamento com 2 dormitorios para locacao  jardim la salle  toledo   pr
6. Coletado: casa para locacao  vila becker  toledo   pr
7. Coletado: casa  03 quarto para locacao  excelente vila becker   toledo pr
8. Coletado: sobrado com 01 suite e 01 quarto para locacao no centro de toledo pr
9. Coletado: casa nova com 03 quartos para locacao  jd  sao francisco   toledo pr
10. Coletado: apartamento com 01 suite e 02 quartos para locacao no centro   toledo pr
11. Coletado: apartamento semi mobiliado para locacao  vila industrial   toledo pr
12. Coletado: apartamento com 2 dormitorios para locacao  vila ind

In [41]:
dados_coletados_geral = persist(dados_coletados_geral)

# Imobiliaria Lokatell-Bachman

In [ ]:
print(f'{azul}------------------Iniciado Imobiliária Lokatell------------------{fim}\n')
dados_coletados_geral = []
driver = webdriver.Chrome()
url = "https://lokatell.com.br/aluguel/residencial_comercial/toledo/"
driver.get(url)
driver.maximize_window()

wait = WebDriverWait(driver, 15)
container = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "clb-search-result-property")))

prev_count = 0
same_count_times = 0
contador_imoveis = 0
links_coletados = set()

while True:
    driver.execute_script("arguments[0].scrollTop += 400;", container)
    time.sleep(1)

    cards = driver.find_elements(By.CLASS_NAME, "imovel-box-single")
    current_count = len(cards)

    for card in cards:
        try:
            titulo = card.find_element(By.CLASS_NAME, "titulo-grid").text.strip()
        except:
            titulo = None

        try:
            link = card.find_element(By.XPATH, ".//div[@class='titulo-anuncio']/a").get_attribute('href')
        except:
            link = None

        if link in links_coletados:
            continue

        try:
            raw_address = card.find_element(By.CSS_SELECTOR, '[itemprop="streetAddress"]')
            address = raw_address.text.strip()

            if '-' in address:
                bairro, city_state = address.split('-', 1)
                bairro = bairro.strip()
            else:
                bairro = None
        except:
            bairro = None

        father = card.find_element(By.CLASS_NAME, 'property-amenities.amenities-main')
        blocks = father.find_elements(By.XPATH, './div')
        quartos = suites = area = 0

        try:
            for block in blocks:
                label = block.find_element(By.TAG_NAME, 'small').text.strip().lower()
                if 'suíte' in label or 'suítes' in label:
                    raw_suite= block.find_element(By.TAG_NAME, 'span').text.strip()
                    m = re.search(r'\d+', raw_suite)
                    suites = int(m.group(0)) if m else 0
                    break
        except:
            suites = 0

        try:
            for block in blocks:
                label = block.find_element(By.TAG_NAME, "small").text.strip().lower()
                if "quarto" in label or "quartos" in label:
                    raw_bedroom = block.find_element(By.TAG_NAME, "span").text.strip()
                    m = re.search(r'\d+', raw_bedroom)
                    quartos = int(m.group(0)) if m else 0
                    break
        except:
            quartos = 0

        try:
            for block in blocks:
                raw_area = block.find_element(By.TAG_NAME, "span").text.strip()
                if "m²" in raw_area:
                    m = re.search(r'[\d\.,]+', raw_area)
                    if m:
                        num_s = m.group(0).replace('.', '').replace(',', '.')
                        try:
                            area = float(num_s)
                        except:
                            area = 0
                    else:
                        area = 0
                    break
        except:
            area = 0

        try:
            raw_price = card.find_element(By.CSS_SELECTOR, '[itemprop="price"]').text.strip()
            m = re.search(r'[\d\.,]+', raw_price)
            if m:
                num_s = m.group(0).replace('.', '').replace(',', '.')
                try:
                    preco = float(num_s)
                except:
                    preco = 0
        except:
            preco = 0

        try:
            condominio = int(card.find_element(By.CLASS_NAME, "item-price-condominio").text.strip().split()[-1].split(',')[0])
        except:
            condominio = 0
        
        tipo = None
        if "Barracão" in titulo:
            tipo = "Barracão"
        elif "Terreno" in titulo:
            tipo = 'Terreno'
        elif "Casa" or "Sobrado" or "Apartamento" in titulo:
            tipo = "Residencial"
        elif "Loja" or "Sala Comercial" in titulo:
            tipo = "Sala Comercial"
        else:
            tipo = "Não Identificado"

        registro = {
            "Titulo do imóvel": titulo,
            "Tipo do imóvel": tipo, 
            "Preço": preco,
            "Valor Condomínio": condominio,
            "Bairro": bairro,
            "Link para o anúncio": link,
            "Origem": "Lokatell",
            "Quartos": quartos,
            "Suítes": suites,
            "Área do imóvel": area,
        }


        if registro not in dados_coletados_geral:
            dados_coletados_geral.append(registro)
            links_coletados.add(link)
            contador_imoveis += 1
            print(f"{contador_imoveis}.{verde} Coletado: {titulo}{fim}")

    # controle de fim da rolagem
    if current_count == prev_count:
        same_count_times += 1
    else:
        same_count_times = 0

    if same_count_times >= 8:
        print(f"{amarelo}Nenhum novo imóvel carregado, finalizando scroll.{fim}")
        break

    prev_count = current_count

print(f"{azul}Total de imóveis coletados: {contador_imoveis}{fim}\n")

print(f'\n{azul}------------------Finalizado Imobiliária Lokatell------------------{fim}')
driver.close()


------------------Iniciado Imobiliária Lokatell------------------

1. Coletado: Casa 2 Quartos Sao Francisco 56m²
2. Coletado: Apartamento 3 Quartos Jardim La Salle 80m²
3. Coletado: Casa 3 Quartos Jardim Panorama 75m²
4. Coletado: Casa 3 Quartos Vila Industrial 80m²
5. Coletado: Apartamento 2 Quartos Jardim Coopagro 60m²
6. Coletado: Casa 4 Quartos Jardim Pancera 343m²
7. Coletado: Casa 2 Quartos Jardim Gisela 70m²
8. Coletado: Apartamento 2 Quartos Vila Industrial 70m²
9. Coletado: Apartamento 2 Quartos Jardim Coopagro 56m²
10. Coletado: Casa 2 Quartos Jardim Coopagro 63m²
11. Coletado: Casa 3 Quartos Jardim Panorama 200m²
12. Coletado: Sobrado 3 Quartos Jardim Pancera 80m²
13. Coletado: Apartamento 2 Quartos Jardim Pancera 65m²
14. Coletado: Casa 3 Quartos Jardim Parizzotto 72m²
15. Coletado: Casa 2 Quartos Jardim Porto Alegre 50m²
16. Coletado: Apartamento 2 Quartos Sao Francisco 55m²
17. Coletado: Casa 3 Quartos Jardim Pancera 105m²
18. Coletado: Casa 2 Quartos Jardim Coopagro 70m

In [43]:
dados_coletados_geral = persist(dados_coletados_geral)

# Imobiliaria Ativa-Angelo

In [4]:
print(f'{azul}------------------Iniciado Imobiliária Ativa------------------{fim}\n')

class_imovel = 'muda_card1.ms-lg-0.col-12.col-md-12.col-lg-6.col-xl-4.mt-4.d-flex.align-self-stretch.justify-content-center'

def getPage(n):
    return f'https://www.imobiliariaativa.com.br/pesquisa-de-imoveis/?locacao_venda=L&finalidade=&dormitorio=&garagem=&vmi=&vma=&ordem=6&&pag={n}'

def percorra(driver):
    have = True
    locs = 0
    pag = 0
    while have:
        try:
            pag += 1
            driver.get(getPage(pag))
            imoveis = driver.find_elements(By.CLASS_NAME, class_imovel)
            if pag>10:
                print('limite atingido')
                return
            if locs >= int(driver.find_element(By.XPATH,'/html/body/main/section/div/h1').text.split(' ')[0]):
                return
            yield imoveis
        finally:
            pass

def calcularTipo(titulo):
    lTitle = titulo.lower()
    if 'casa' or 'sobrado' or 'apartamento' in lTitle:
        return 'Residencial'
    elif 'comercia' in lTitle:
        return 'Sala Comercial'
    elif 'barracão' in lTitle:
        return 'Barracão'
    elif 'Terreno' in lTitle:
        return 'Terreno'
    else:
        return 'Não Identificado'

def buscarCondominio(link, driver, bustarArea, area, tipo):
    driver.execute_script(f"window.open('{link}');")
    driver.switch_to.window(driver.window_handles[1])
    condominio = 'None'
    
    if tipo == 'Residencial':
        try:
            for i in range(2, 10):
                if driver.find_element(By.XPATH,f'//*[@id="valores_imovel"]/div[{i}]/div[1]').text ==  'Condomínio':
                    condominio = int(driver.find_element(By.XPATH,f'//*[@id="valores_imovel"]/div[{i}]/div[2]').text.split(',')[0])
                    break
                else:
                    condominio = 0
        except:
            print('', end = '')
            
    if bustarArea:
        area = [area for area in [e.text for e in driver.find_elements(By.CLASS_NAME,'fw-bold')] if ' m²' in area]
        if len(area) > 0:
            area = float(area[0].split()[0])
        else:
            descricao = driver.find_element(By.CLASS_NAME,'descricao-imovel').text
            if ' m²' in descricao:
                palavras = descricao.split()
                for i, palavra in enumerate(palavras):
                    if palavra == 'm²':
                        area = float(f'{palavras[i-1].replace(',','.')} {palavras[i]}'.split()[0])
                        
            else:
                area = None

    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    return (condominio, area)

def getAttrs(imovel, driver, contador):
    titulo = imovel.find_element(By.CLASS_NAME,'card-titulo.corta-card-titulo').text.replace(',','.')
    tipo = calcularTipo(titulo)
    link = imovel.find_element(By.CLASS_NAME,'carousel-cell.is-selected').get_attribute('href')
    buscarArea = False
    suites = ''
    quartos = ''
    area = ''
    condominio = 'None'
    try:
        extras  = imovel.find_element(By.CLASS_NAME,'imo-dad-compl')
        for extra in extras.find_elements(By.TAG_NAME,'div'):
            if 'dorm-ico' in extra.get_attribute('class').split():
                quartos = extra.find_elements(By.TAG_NAME,'span')[1].text
            else:
                quartos = 0
            if 'suites-ico' in extra.get_attribute('class').split():
                suites = extra.find_elements(By.TAG_NAME,'span')[1].text
            else:
                suites = 0
            if 'a-terr-ico' in extra.get_attribute('class').split():
                area = extra.find_elements(By.TAG_NAME,'span')[1].text
            elif 'a-const-ico' in extra.get_attribute('class').split():
                area = extra.find_elements(By.TAG_NAME,'span')[1].text
            elif 'a-util-ico' in extra.get_attribute('class').split():
                area = extra.find_elements(By.TAG_NAME,'span')[1].text
            elif 'a-total-ico' in extra.get_attribute('class').split():
                area = extra.find_elements(By.TAG_NAME,'span')[1].text
            else:
                buscarArea = True
    except:
        print(f'{vermelho}Não foi possível alguns dados do imóvel{fim}')

    try:
        valor = int(imovel.find_element(By.CLASS_NAME,'card-valores').text.split(',')[0].replace('.', '').split()[1])
    except:
        valor = 0
        
    condominio, area = buscarCondominio(link, driver, buscarArea, area, tipo)

    bairro = imovel.find_element(By.CLASS_NAME,'card-bairro-cidade-texto').text.split('-')[0][:-1]
    print(f"{contador}.{verde} Coletado:", titulo, f'{fim}')

    registro = {
    "Titulo do imóvel": titulo,
    "Tipo do imóvel": tipo,
    "Preço": valor,
    "Valor Condomínio": condominio,
    "Bairro": bairro,
    "Link para o anúncio": link,
    "Origem": "Ativa",
    "Quartos": quartos,
    "Suítes": suites,
    "Área do imóvel": area,
}


    return registro
    
driver = webdriver.Chrome()
driver.maximize_window()

contador_imoveis = 1
for imoveis in percorra(driver):
    for imovel in imoveis:
        registro = getAttrs(imovel, driver, contador_imoveis)
        contador_imoveis += 1
        dados_coletados_geral.append(registro)

driver.quit()

print(f'{azul}------------------Finalizado Imobiliária Ativa------------------{fim}')

print(f"Foram coletados ao todo {vermelho}{len(dados_coletados_geral)}{fim} imóveis")

print(f'{verde}Preparando salvamento...{fim}')

# dados_coletados_geral = persist(dados_coletados_geral)



------------------Iniciado Imobiliária Ativa------------------

1. Coletado: Casa - Centro 
2. Coletado: Apartamento - Padrão 
3. Coletado: Casa - Padrão 
4. Coletado: Casa Padrão com otima localização Jardim Pancera 
5. Coletado: Casa Nova com otimo espaço. Jardim São Francisco 
6. Coletado: Casa - Padrão 
7. Coletado: Apartamento - Padrão 
8. Coletado: Casa - Sobrado 
9. Coletado: Apartamento - Padrão 
10. Coletado: Casa - Padrão 
11. Coletado: Apartamento - Padrão 
12. Coletado: Apartamento Padrão localizado no Bairro São Francisco 
13. Coletado: Casa com 1 suite. 1 Quarto e 1 Espaço Integrada aos fundos- Ideal para Profissionais Autônomos 
14. Coletado: CASA ALTO PADRAO NO JARDIM LA SALLE 
15. Coletado: Apartamento - Padrão 
16. Coletado: Apartamento padrão Localizado no BIOPARK 
17. Coletado: Apartamento - Padrão 
18. Coletado: Apartamento - Padrão 
19. Coletado: Aluguel no Jardim La Salle - Casa com Piscina e 4 Quartos! 
20. Coletado: Apartamento padrão com suíte localizado no Ce

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: Unable to receive message from renderer
  (Session info: chrome=140.0.7339.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
#0 0x59ea87844fba <unknown>
#1 0x59ea872c96d0 <unknown>
#2 0x59ea872b19ae <unknown>
#3 0x59ea872b16b5 <unknown>
#4 0x59ea872af53b <unknown>
#5 0x59ea872afe8f <unknown>
#6 0x59ea872cfcf7 <unknown>
#7 0x59ea872d2be8 <unknown>
#8 0x59ea872d2c73 <unknown>
#9 0x59ea8731583d <unknown>
#10 0x59ea87340e32 <unknown>
#11 0x59ea8730ef47 <unknown>
#12 0x59ea87340ffe <unknown>
#13 0x59ea87366b54 <unknown>
#14 0x59ea87340c03 <unknown>
#15 0x59ea8730d7a8 <unknown>
#16 0x59ea8730e421 <unknown>
#17 0x59ea87809b28 <unknown>
#18 0x59ea8780d87f <unknown>
#19 0x59ea877f1c49 <unknown>
#20 0x59ea8780e405 <unknown>
#21 0x59ea877d74ff <unknown>
#2

# Imobilia Maximize-Angelo

In [46]:
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/128.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.5 Safari/605.1.15',
]

dados_coletados_geral = []

print(f'{azul}\n------------------Iniciado Imobiliária Maximize------------------{fim}')
contador_imoveis = 0
pagina = 1



def getattr(link,headers):
    
    html_content = requests.get(link, headers = headers, timeout=20).text
    response = scrapy.http.HtmlResponse(url = link, body = html_content, encoding = "utf-8")
    
    preco = response.xpath(
    '//span[contains(text(), "Locação")]/following-sibling::p[1]/text()'
    ).get()
    if not preco:
        preco = response.xpath(
        '//div[contains(text(), "Locação")]/following-sibling::p[1]/text()'
        ).get() 


    quartos = response.xpath(
        '//div[contains(@class,"big-card-property-facility-exebicao")][div[contains(text(),"Quartos")]]//div[contains(@class,"text-r-bold-exebicao")]/text()'
    ).get()
    if not quartos:
        quartos = response.xpath(
        '//div[contains(@class,"big-card-property-facility-exebicao")][div[contains(text(),"Quarto")]]//div[contains(@class,"text-r-bold-exebicao")]/text()'
    ).get()

    suites = response.xpath(
        '//div[contains(@class,"big-card-property-facility-exebicao")][div[contains(text(),"Suítes")]]//div[contains(@class,"text-r-bold-exebicao")]/text()'
    ).get()
    if not suites:
        suites = response.xpath(
            '//div[contains(@class,"big-card-property-facility-exebicao")][div[contains(text(),"Suíte")]]//div[contains(@class,"text-r-bold-exebicao")]/text()'
        ).get()
    area = response.xpath(
        '//div[contains(@class,"big-card-property-facility-exebicao")][div[contains(text(),"Área Total")]]//div[contains(@class,"text-r-bold-exebicao")]/text()'
    ).get()

    if not area:
        area = response.xpath(
            '//div[contains(@class,"big-card-property-facility-exebicao")][div[contains(text(),"Área Privativa")]]//div[contains(@class,"text-r-bold-exebicao")]/text()'
        ).get()
    if not area:
        area = response.xpath(
            '//div[contains(@class,"big-card-property-facility-exebicao")][div[contains(text(),"Área Construída")]]//div[contains(@class,"text-r-bold-exebicao")]/text()'
        ).get()

    if not area:
        area = response.xpath(
            '//div[contains(@class,"big-card-property-facility-exebicao")][div[contains(text(),"Área Terreno")]]//div[contains(@class,"text-r-bold-exebicao")]/text()'
        ).get()
    if not area:
        descriacao = response.xpath(
        '//*[@id="w-node-_501b00e3-5969-2d2f-4246-33f42ad8c0cc-51cf1fd3"]/div[2]/div[2]/p'
        ).get()
        descriacao = descriacao.split('<br>')
        for part in descriacao:
            if 'm²' in part:
                # print(part)
                area = part.split(':')[-1].replace('.','').split(' ')
                for i,a in enumerate(area):
                    if 'm²' in a:
                        # area = a
                        if a == 'm²':
                            try:
                                area = area[i-1]+a
                            except:
                                area = ''
                        else:
                            area = a
                break
        # print(descriacao)
    condominio = response.xpath(
        '//span[contains(text(), "Valor de Condominio:")]/following-sibling::p[1]/text()'
        ).get()



    return (preco,condominio,quartos,suites,area)

for pagina in range(1,6):
    print(f"{azul}Coletando página: {pagina}{fim}")
    url_geral = f'https://www.maximizeimobiliaria.com.br/filtro2/list/locacao/todos/todas/todos/0-10000000/todos/{pagina}/ASC'
    random_user_agent = random.choice(user_agents)
    headers = {"User-Agent": random_user_agent}

    try:
        html_content = requests.get(url_geral, headers = headers, timeout=20).text
    except (requests.exceptions.ReadTimeout, requests.exceptions.ConnectionError) as e:
        print(f"{amarelo}Erro de conexão/timeout na página {pagina}. Tentando novamente em 7 segundos...{fim}")
        time.sleep(7)
        continue

    # time.sleep(random.uniform(2, 5))
    response = scrapy.http.HtmlResponse(url = url_geral, body = html_content, encoding = "utf-8")

    imoveis_na_pagina = response.css("div.card-property-exebicao.raw-exebicao.w-dyn-item")
    if len(imoveis_na_pagina) == 0:
        break

    for imovel in imoveis_na_pagina:
        contador_imoveis += 1

        titulo = imovel.css("h5.heading-2-exebicao::text").get()

        tipo = None
        if "Apartamento" or "Casa" or "Sobrado" in titulo:
            tipo = "Residencial"
        elif "Barracão" in titulo:
            tipo = "Barracão" 
        elif "Sala" in titulo:
            tipo = "Sala Comercial"
        else:
            tipo = "Não identificado"

        link = imovel.css("div.slide-exebicao.w-slide>a::attr(href)").get()

        preco,condominio,quartos,suites,area = getattr(link,headers)

        dados_coletados_geral.append({
            "Titulo do imóvel": titulo,
            "Tipo do imóvel": tipo,
            "Preço": preco,
            "Valor Condomínio": condominio,
			"Bairro": imovel.css("div.card-property-description-exebicao>div::text").get(),
    		"Link para o anúncio": link,
            "Origem": 'Maximize Imobiliária',
            "Quartos": quartos,
            "Suítes": suites,
            "Área do imóvel": area
        })


        print(f"{contador_imoveis}.{verde} Coletado:", titulo, f'{fim}')
        # time.sleep(0.5)
    # time.sleep(1)
    # pagina += 1
    # break

print(f'\n{azul}------------------Finalizado Imobiliária Maximize------------------{fim}')

persist(dados_coletados_geral)


# with open("dados_scraper.csv", "w", newline = "", encoding = "utf-8-sig") as f:
#     writer = csv.writer(f)
#     writer.writerow(["Titulo do imóvel", "Tipo", "Preço", 'Valor Condominio', "Quartos", "Suítes", 'Área Total', "Bairro", "Link para o anúncio", 'Origem'])
#     for dado in dados_coletados_geral:
#         writer.writerow([dado["Titulo do imóvel"], dado["Tipo do imóvel"], dado["Preço"], dado["Valor Condomínio"], dado["Quartos"], dado["Suítes"], dado["Área do imóvel"], dado["Bairro"], dado["Link para o anúncio"], dado['Origem']])
#     print(f"{verde}Salvo em CSV!{fim}")



------------------Iniciado Imobiliária Maximize------------------
Coletando página: 1
1. Coletado: Sobrado à venda, Centro, TOLEDO - PR 
2. Coletado: Barracão, 338 m² - venda por R$ 1.350.000,00 ou aluguel por R$ 6.000,00/mês - Ja 
3. Coletado: Sobrado para venda e locação, no Jardim Pancera, com uma suíte e mais dois quartos, piscina, 
4. Coletado: Casa para locação e venda com 03 quartos e piscina, Vila Becker, TOLEDO - PR 
5. Coletado: Apartamento semi-mobiliado  à venda e locação no  Centro de Toledo 
6. Coletado: Apartamento à venda, Jardim La Salle, TOLEDO - PR 
7. Coletado: Apartamento à venda, Jardim Gisela, TOLEDO - PR 
8. Coletado: Sala, 26 m² - venda por R$ 500.000,00 ou aluguel por R$ 2.500,00/mês - Centro -  
9. Coletado: Sala Comercial para locação, Centro, TOLEDO - PR 
10. Coletado: Sala Comercial para locação, Centro, TOLEDO - PR 
11. Coletado: Sala Comercial para locação no Centro de Toledo - PR 
12. Coletado: SALA COMERCIAL COM 220,00 M  - 01 VAGA DE GARAGEM - COPA -

[]